# GenAI with RAG by `Mr. Harshit Dawar!`

In [1]:
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_community.llms import Bedrock

In [2]:
def embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    return embeddings

In [3]:
loader = CSVLoader("Meal Plan for AWS Bedrock.csv")
documents = loader.load()

In [8]:
# documents

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
spliteed_documents = text_splitter.split_documents(documents)

In [7]:
# spliteed_documents

In [9]:
vector_db = Chroma(persist_directory="vectors_directory", embedding_function=embedding_function())
vector_db.add_documents(documents)

['c41ca1a0-dbee-4359-9b63-5cc9e1446600',
 'e09e3b29-e84d-4253-82f5-15d70c811753',
 '14852d61-2cf8-47dc-887d-a46d7f7f95e0',
 'bcaff0e8-b19c-4c31-9e37-a5920a5254b2',
 'b7ed825d-6243-454d-a1a3-4e5c55bb39f2',
 'd4c3fa9c-b215-4b5d-a8be-627515fa580a',
 '690a3e79-0491-4031-9f14-177343da8d15']

In [10]:
vector_db.persist()

In [11]:
llm = Bedrock(
    credentials_profile_name="default", model_id="amazon.titan-text-express-v1", region_name="us-east-1", model_kwargs={"temperature": 0},
    streaming=True
)

C:\Users\HarshitDawar\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


In [12]:
retriever = vector_db.as_retriever()
qa = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [14]:
# qa.invoke("query")

In [15]:
query = "Tell me complete meal plan for Friday"
query1 = "Tell me complete meal plan for Friday, and provide me the input for all the meals that is breakfast, lunch, and dinner, each in the new line."

In [17]:
print(qa.invoke(query)["answer"])


Masala Oats, Vegetable pasta with a tomato-based sauce, Lemony Tofu Stir-fry with a side of quinoa.



In [18]:
print(qa.invoke(query1)["answer"])



Breakfast: Masala Oats
Lunch: Vegetable pasta with a tomato-based sauce
Dinner: Lemony Tofu Stir-fry with a side of quinoa

Breakfast: Vegetable Poha
Lunch: Dal Tadka with brown rice and a side of spinach raita
Dinner: Stuffed bell peppers with a paneer and vegetable filling




In [21]:
query2 = "what is in dinner on Sunday? Prefix the answer with a lot of '*'"

In [22]:
print(qa.invoke(query2)["answer"])

 Stuffed bell peppers with a paneer and vegetable filling.

